In [1]:
!pip install tokenizers

     |████████████████████████████████| 6.8 MB 8.1 MB/s 


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import torch
import torch.nn as nn

# Transformer로 챗봇을 만들어봅시다!
- Transformer에 문장을 입력하면, 그에 대한 대답을 하는 챗봇을 만들 것입니다.
- 중간중간 빈칸을 채워넣어가며 자신만의 챗봇 코드를 완성해보세요.

우선 챗봇 학습을 위한 데이터를 다운로드 받습니다.

In [3]:
# data download
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [4]:
print('데이터 샘플의 개수 :', len(train_data))

데이터 샘플의 개수 : 11823


In [5]:
train_data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


## 한국어 데이터를 전처리하는 방법에 대해 알아봅시다.
- 우리는 네이버 영화리뷰 데이터에 tokenizer를 학습시켜 단어를 구분하도록 할 예정입니다. 
(챗봇 데이터에 비해 훨씬 많은 데이터를 포함하고 있습니다.)

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [7]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(lowercase=False) # lowercase를 구분할지 여부를 선택합니다 (True: 대문자 무시(모두 소문자로 인식), False:대소문자 구분)


In [8]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency,
                special_tokens=['[PAD]', '[START]', '[END]', '[UNK]'])

In [9]:
# vocab 저장
tokenizer.save_model('./')
vocab_df = pd.read_fwf('vocab.txt', header=None)
vocab_df

,0
0,[PAD]
1,[START]
2,[END]
3,[UNK]
4,!
...,...
29995,망하지
29996,망한다
29997,망가지
29998,망정이지


In [10]:
# 챗봇 모델은 "정수 인코딩" 결과를 사용하여 모델을 작동시키고, 최종 예측 결과도 정수로 인코딩 된 결과를 내뱉습니다.
# 인코딩된 결과물을 decode 를 통해 해석 가능한 문장으로 바꿔줄 수 있습니다.
encoded = tokenizer.encode('챗봇이 잘 완성될까요?')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['챗', '##봇', '##이', '잘', '완성', '##될까', '##요', '?']
정수 인코딩 : [2644, 4191, 3263, 2361, 13647, 20331, 3305, 34]
디코딩 : 챗봇이 잘 완성될까요?


# Transformer 모델을 만들어봅시다.

In [39]:
from torch import Tensor
import torch
import torch.nn as nn
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2) # (max length, 1, embedding size)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        # ============== TODO ======================
        # torch.nn 에는 Transformer 모델이 이미 구현되어 있습니다. 
        # 이것을 이용해 챗봇 모델을 완성해봅시다.
        # ==========================================
        self.transformer = nn.Transformer(d_model=emb_size,
                                nhead=nhead,
                                num_encoder_layers=num_encoder_layers,
                                num_decoder_layers=num_decoder_layers,
                                dim_feedforward=dim_feedforward,
                                dropout=dropout)
        # Transformer의 output값을 vocabulary의 index를 나타내는 벡터로 바꾸어주는 선형 변환이 필요합니다.
        self.generator = nn.Linear(emb_size, tgt_vocab_size) 
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        # ============== TODO ======================
        # 위에서 구현한 트랜스포머 모델에 입력된 데이터를 통과시켜봅시다.
        # ==========================================
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

- Transformer의 학습 중에는, 미래의 데이터를 볼 수 없도록 하는 마스크가 필요합니다.

In [12]:
PAD_IDX = 0
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [15]:
SRC_VOCAB_SIZE = vocab_size # 30000
TGT_VOCAB_SIZE = vocab_size # 30000
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# 모델을 initialize 해줍니다.
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model = model.to(DEVICE) # model 을 GPU로 보내줍니다.

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX) # ignore_index 가 하는 역할은 무엇일까요? -> 해당 index의 data를 loss에 이용하지 않음

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [24]:
from torch.utils.data import Dataset
class ChatbotDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y, _ = self.data[idx]
        # ============== TODO ==============
        # 1. 데이터를 토크나이즈 해보세요. (x,y 모두)
        # 2. start token 과 end token을 추가해보세요. (x,y 모두)
        # 3. 토큰들을 torch tensor (long 타입)로 변환하고 return하세요.
        # ==================================

        x_tokens = self.tokenizer.encode(x).ids
        y_tokens = self.tokenizer.encode(y).ids

        # hint: x_tokens 와 y_tokens 는 list type입니다.
        x_tokens.insert(0, 1)
        x_tokens.append(2)
        y_tokens.insert(0, 1)
        y_tokens.append(2)

        return torch.LongTensor(x_tokens), torch.LongTensor(y_tokens)

In [25]:
dataset = ChatbotDataset(train_data.values, tokenizer)

In [26]:
x, y = dataset[0]

In [28]:
x, y

(tensor([   1, 7335, 3281, 1211,    4,    2]),
 tensor([    1, 28332,  1240, 18157,    17,     2]))

In [27]:
tokenizer.decode(x.tolist())

'12시 땡!'

In [29]:
tokenizer.decode(y.tolist())

'하루가 또 가네요.'

In [24]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [30]:
# collate_fn 을 정의해줍니다.
# collate_fn 은 dataset으로부터 여러 item을 받아와 하나의 배치로 합칠 때, 어떻게 합칠지를 정의하는 부분입니다.


def collate_fn(batch):
    # batch: [(x1, y1), (x2, y2), ... ]
    # ============== TODO ==============
    # 우리의 목표는 X = [x1,x2,x3,...] , Y = [y1, y2, y3, ...] 형태의 tensor로 만드는 것입니다.
    # 이를 위해 2가지 해야할 일이 있습니다.
    # 1. batch 안에 있는 x 와 y를 각각의 list 에 모아주기.
    # 2. padding을 통해 동일한 길이로 만들어주고, 하나의 tensor로 통합하기.
    # 아래 빈칸을 채워 위 두가지를 진행해보세요.
    # ==================================
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)
    
    # padding_value는 pad 위치에 어떤 값을 넣을지를 정하는 값입니다.
    src_batch = pad_sequence(src_batch, padding_value=0)
    tgt_batch = pad_sequence(tgt_batch, padding_value=0) 
    return src_batch, tgt_batch

# dataloader 를 정의해줍니다.
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=collate_fn,
                          pin_memory=True, drop_last=True)

In [31]:
x,y = next(iter(data_loader))

In [32]:
x.shape, y.shape

(torch.Size([15, 32]), torch.Size([15, 32]))

In [33]:
x[:,1]

tensor([   1, 2010, 3347, 6769,    2,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0])

In [34]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, loss_fn, train_dataloader):
    model.train() # dropout 층의 적용유무가 달라짐
    losses = 0

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1,:] # 왜 target은 마지막 하나를 빼고 입력할까요?

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :] # 왜 첫번째 단어는 빼고 로스를 계산할까요?
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)

In [35]:
EPOCHS = 10
for i in range(EPOCHS):
    epoch_loss = train_epoch(model, optimizer, loss_fn, data_loader)
    print('EPOCH {} LOSS {:.6f}'.format(i+1, epoch_loss))

EPOCH 1 LOSS 6.110334
EPOCH 2 LOSS 5.194339
EPOCH 3 LOSS 4.803519
EPOCH 4 LOSS 4.477032
EPOCH 5 LOSS 4.195782
EPOCH 6 LOSS 3.944159
EPOCH 7 LOSS 3.708910
EPOCH 8 LOSS 3.493097
EPOCH 9 LOSS 3.286914
EPOCH 10 LOSS 3.091884


In [36]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == 2:
            break
    return ys

def predict(model, tokenizer, src_sentence):
    model.eval()
    src = tokenizer.encode(src_sentence).ids
    src.insert(0,1)
    src.append(2)
    src = torch.tensor(src).long().unsqueeze(1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=1).flatten()
    return tokenizer.decode(tgt_tokens.cpu().tolist())

In [37]:
predict(model,tokenizer,'반가워')

'저도요.'

In [38]:
predict(model,tokenizer,'여행 가고 싶다.')

'저도요!'

In [43]:
predict(model,tokenizer,'오늘 치킨 어때?')

'저는 위로봇입니다.'

# BERT 실습
- BERT는 Transformer의 encoder를 사용합니다.
- Transformer 구현에 대해 이미 알아보았으니, BERT를 직접 구현하지 않고 huggingface 라이브러리를 통해 간단하게 구현하는 방법에 대해 배우도록 하겠습니다.
- BERT를 이용해서 naver 영화리뷰 데이터의 점수를 분류하는 task를 진행해보겠습니다.
- https://huggingface.co/

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 61 kB 498 kB/s 
     |████████████████████████████████| 596 kB 47.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#### 영화리뷰 데이터에 대한 fine-tuning에 앞서, 간단하게 huggingface 라이브러리의 사용방법에 대해 익혀보도록 합시다.

- https://huggingface.co/transformers/v3.0.2/index.html

In [2]:
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [3]:
outputs

TokenClassifierOutput([('logits', tensor([[[-0.3753, -0.7052],
                                 [-0.8380, -0.6384],
                                 [-0.6111, -0.5653],
                                 [-0.6218, -0.4332],
                                 [-0.0795, -0.1445],
                                 [-0.7203, -0.6385],
                                 [-0.2776,  0.0162],
                                 [ 0.0530, -0.2358]]], grad_fn=<AddBackward0>))])

In [4]:
inputs.shape

torch.Size([1, 8])

- 사전 학습된 BERT의 mask 토큰에 대한 예측 결과를 확인해봅시다.

In [5]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
unmasker("The man worked as a [MASK].")

[{'score': 0.09747546911239624,
  'sequence': 'the man worked as a carpenter.',
  'token': 10533,
  'token_str': 'carpenter'},
 {'score': 0.052383411675691605,
  'sequence': 'the man worked as a waiter.',
  'token': 15610,
  'token_str': 'waiter'},
 {'score': 0.04962698742747307,
  'sequence': 'the man worked as a barber.',
  'token': 13362,
  'token_str': 'barber'},
 {'score': 0.037886083126068115,
  'sequence': 'the man worked as a mechanic.',
  'token': 15893,
  'token_str': 'mechanic'},
 {'score': 0.037680838257074356,
  'sequence': 'the man worked as a salesman.',
  'token': 18968,
  'token_str': 'salesman'}]

In [7]:
unmasker("The woman worked as a [MASK].")

[{'score': 0.21981535851955414,
  'sequence': 'the woman worked as a nurse.',
  'token': 6821,
  'token_str': 'nurse'},
 {'score': 0.1597413569688797,
  'sequence': 'the woman worked as a waitress.',
  'token': 13877,
  'token_str': 'waitress'},
 {'score': 0.11547300964593887,
  'sequence': 'the woman worked as a maid.',
  'token': 10850,
  'token_str': 'maid'},
 {'score': 0.03796879202127457,
  'sequence': 'the woman worked as a prostitute.',
  'token': 19215,
  'token_str': 'prostitute'},
 {'score': 0.030423851683735847,
  'sequence': 'the woman worked as a cook.',
  'token': 5660,
  'token_str': 'cook'}]

In [8]:
unmasker("I have a [MASK].")

[{'score': 0.0911906436085701,
  'sequence': 'i have a plan.',
  'token': 2933,
  'token_str': 'plan'},
 {'score': 0.04680357500910759,
  'sequence': 'i have a problem.',
  'token': 3291,
  'token_str': 'problem'},
 {'score': 0.030351677909493446,
  'sequence': 'i have a girlfriend.',
  'token': 6513,
  'token_str': 'girlfriend'},
 {'score': 0.029900116845965385,
  'sequence': 'i have a point.',
  'token': 2391,
  'token_str': 'point'},
 {'score': 0.027309242635965347,
  'sequence': 'i have a boyfriend.',
  'token': 6898,
  'token_str': 'boyfriend'}]

- BERT tokenizer와 model을 이용해 네이버 영화리뷰 데이터에 fine-tuning을 진행해봅시다.

In [11]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [12]:
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [13]:
naver_df.tail()

,id,document,label
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0
199999,8548411,포풍저그가나가신다영차영차영차,0


In [14]:
naver_df['label'].unique()

array([1, 0])

In [15]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') # multilingual-BERT를 사용해보겠습니다.
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [16]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[-0.0750,  0.0802]], grad_fn=<AddmmBackward0>))])

In [17]:
# label을 입력해주면 classification에 대한 loss도 자동으로 계산할 수 있습니다.
model(inputs, labels=torch.tensor([1]))

SequenceClassifierOutput([('loss', tensor(0.6185, grad_fn=<NllLossBackward0>)),
                          ('logits',
                           tensor([[-0.0750,  0.0802]], grad_fn=<AddmmBackward0>))])

In [18]:
train_data_idx = np.random.choice(range(len(naver_df)), size=len(naver_df)//5*4, replace=False)

In [19]:
train_data_idx

array([ 32750, 131035,  63209, ...,  96485,  79929,  39884])

In [20]:
train_data = naver_df.iloc[train_data_idx][['document','label']].values
test_data = naver_df.iloc[~naver_df.index.isin(train_data_idx)][['document','label']].values

In [21]:
from torch.utils.data import Dataset
class ReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        # BERT tokenizer는 batch 단위로 한번에 token화 할 수 있습니다.
        # dataloder에서 얻어온 후 한번에 tokenize 해보도록 하겠습니다.
        # text는 list로, label은 long tensor로 리턴해줍니다.
        return x, torch.tensor(y).long()

In [22]:
train_dataset = ReviewDataset(train_data)
test_dataset = ReviewDataset(test_data)

In [25]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=False)

In [26]:
x, y = next(iter(train_loader))

In [27]:
x

['하 영화보면서 울었던 적은 처음... 진심 이런 개와 주인 사랑합니다.개가 역시 사람보다 나을떄도 있네요..',
 '약간의 사건과 일상. 길다.',
 '뭐야이거...이상해......;;;;;;;;;;;;;;;;',
 '시덥잖은 철학은 니 혼자 알고있어라',
 'ㅋㅋㅋ 초등학교때보고 성인이되서도 재밌게보는영화!!상상력이대단하다 ㅋㅋㅋ',
 'ㅅㅂ 내돈',
 '환경의 소중함을 느꼇음',
 '...전지현만 아니였어도 욕나오는 영화네요..시간아깝다 정말..',
 '그럴듯해보이는 cg효과가 있어 3점',
 '롭코헨 감독 영화중 이영화 하나 쓸만하다 최고의 영화중 하나',
 '에휴~신은경 한물가꾸나..ㅉ',
 '신나게 놀다가 어쩔수없이 총을 겨누고 전쟁하는장면에서 울컥햇다 박찬욱 영화중최고인거같다',
 '이소연 리즈시절 결혼축하',
 '아 근데뭔가재미있음',
 '.스토리조잡하고,박희순연기력도 제대로 못뽑아내고, 이건뭐 그냥 조잡 그자체..',
 '최고의 영화!!! 이렇게 재밌을줄이야><',
 '이걸 보느니 그냥 야동을 봐라',
 '아마4번이상은 볼정도로 이 영화를 정말 좋아했었다 어딘가에 잘 숨어서 행복하기를..',
 '제목도 모르고 로빈윌리엄스가 나올길래 뭔가 싶어 보다 계속봤다.. 멋찐영화!!',
 '나는 솔직히 판의 미로보다 이게 더 재밌었다.',
 '불필요한것에까지욕심을버리지못했던저를반성해보는시간이였습니다.',
 '적절한 센스의 패러디 최고의 영화!',
 '괜찬아.. 볼만해.. 너희도 봐',
 '역시 로코의 여왕 엄정화 쵝오',
 'ㅎㅎ 아나 보는 내내 닭살이 쫙쫙돋았어 이건 뭐 초등학생도 보고 유치하다고 하것어',
 '이거 보느니 롤 일반게임 한번 더 하세요 패배해도 그게 이익입니다',
 '코미디 사극에 장혁의 명품 연기까지...',
 '좋은 소재를 엉망진창의 스토리와, 뻘스러운 연출, 손발퇴갤의 대사, 연기마져도 작품에 흡입력을 흐리게 한다. 정말 좋은 소재였는데 이따위로 망칠 수 있다는게 한심하다. 매력적인 소재를 좋아하는 분은 볼만함. 단 내용에서 뭔가

In [28]:
y

tensor([1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 0])

In [29]:
encoded_x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')

In [30]:
encoded_x.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [31]:
encoded_x['input_ids']

tensor([[  101,  9952, 42428,  ...,     0,     0,     0],
        [  101,  9539, 60454,  ...,     0,     0,     0],
        [  101,  9303, 21711,  ...,     0,     0,     0],
        ...,
        [  101,  8898, 11102,  ...,     0,     0,     0],
        [  101,  9356, 11018,  ...,     0,     0,     0],
        [  101,  9477, 26444,  ...,     0,     0,     0]])

In [32]:
encoded_x['token_type_ids']

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [33]:
encoded_x['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [45]:
def train_epoch(model, dataloader, tokenizer, optimizer):
    model.train()
    train_loss = 0
    for i, (x,y) in enumerate(dataloader):
        # =========== TODO ============
        # 위에서 배운 내용을 바탕으로 빈칸을 채워보세요!
        # =============================
        x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
        y = y.to(DEVICE)
        loss = model(x, labels=y)[0] # model(x)['loss']도 가능 output = (loss, logits)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if i % 50 == 0:
            print('Iter [{}/{}] Loss {:.6f}'.format(i+1, len(dataloader), train_loss / (i+1)))
    
    return train_loss / len(dataloader)

def test_epoch(model, dataloader, tokenizer):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
      for x,y in dataloader:
          # =========== TODO ============
          # 위에서 배운 내용을 바탕으로 빈칸을 채워보세요!
          # =============================
          x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
          out = model(x)[0] # model(x)['logits']도 가능(loss가 없어서 0도 가능)
          pred = out.argmax(-1)
          preds.append(pred.cpu())
          labels.append(y)
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    acc = (preds == labels).float().mean()
    print('ACC : {:.3f}'.format(acc))
    return preds, labels

def predict(model, tokenizer, sentence):
    model.eval()
    x = tokenizer.encode(sentence, return_tensors='pt').to(DEVICE)
    out = model(x)['logits']
    pred = out.argmax(-1)
    return pred.cpu()

In [46]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [47]:
EPOCHS=1

for i in range(EPOCHS):
    train_epoch(model, train_loader, tokenizer, optimizer)
    test_epoch(model, test_loader, tokenizer)

Iter [1/4999] Loss 0.725633
Iter [51/4999] Loss 0.701567
Iter [101/4999] Loss 0.700103
Iter [151/4999] Loss 0.693392
Iter [201/4999] Loss 0.678441
Iter [251/4999] Loss 0.656950
Iter [301/4999] Loss 0.632264
Iter [351/4999] Loss 0.613119
Iter [401/4999] Loss 0.592177
Iter [451/4999] Loss 0.579399
Iter [501/4999] Loss 0.568967
Iter [551/4999] Loss 0.558003
Iter [601/4999] Loss 0.547257
Iter [651/4999] Loss 0.537881
Iter [701/4999] Loss 0.530998
Iter [751/4999] Loss 0.523922
Iter [801/4999] Loss 0.517423
Iter [851/4999] Loss 0.513136
Iter [901/4999] Loss 0.508837
Iter [951/4999] Loss 0.503227
Iter [1001/4999] Loss 0.499305
Iter [1051/4999] Loss 0.494697
Iter [1101/4999] Loss 0.491465
Iter [1151/4999] Loss 0.487537
Iter [1201/4999] Loss 0.483330
Iter [1251/4999] Loss 0.478632
Iter [1301/4999] Loss 0.475814
Iter [1351/4999] Loss 0.472449
Iter [1401/4999] Loss 0.469342
Iter [1451/4999] Loss 0.466787
Iter [1501/4999] Loss 0.464091
Iter [1551/4999] Loss 0.461995
Iter [1601/4999] Loss 0.459779


In [48]:
predict(model, tokenizer, '이 영화는 최고야')

tensor([1])

In [49]:
predict(model, tokenizer, '이 영화는 별로야')

tensor([0])

In [62]:
predict(model, tokenizer, '웹툰의 실사화는 별로라는 편견을 깼다')

tensor([0])

In [55]:
predict(model, tokenizer, '이걸 만든 감독이 대단하다')

tensor([0])

In [66]:
predict(model, tokenizer, '추천해준 친구가 밉다')

tensor([1])